In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os

def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def get_csi_amplitudes(csi_value):
    csi_amplitudes = []
    for v in csi_value:
        amplitude = np.absolute(v)
        csi_amplitudes.append(amplitude)
    return csi_amplitudes

def get_data(mat_filename, transmitter, receiver, authenticated_pair = [' ',' ']):
    annots = loadmat(mat_filename)
    data = []
    for i in range(len(annots['Raw_Cell_Matrix'])):
        authenticated = 0
        if transmitter == authenticated_pair[0] and receiver == authenticated_pair[1]:
            authenticated = 1
        data.append([
            int(annots['Raw_Cell_Matrix'][i][0]['timestamp_low'][0][0][0][0]),
            transmitter,
            receiver,
            annots['Raw_Cell_Matrix'][i][0]['agc'][0][0][0][0]
            ] + get_csi_amplitudes(annots['Raw_Cell_Matrix'][i][0]['CSI'][0][0][0][0])
            + [authenticated]
            )

    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns + ['Authenticated']
    final_data = pd.DataFrame(data, columns=columns)
    return final_data

# print(prepare_data('data.mat','S13','S21'))

from pathlib import Path

def prepare_data(directory, authenticated_pair):
    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns
    all_data = pd.DataFrame([], columns=columns)
    for p in getListOfFiles(Path(directory)):
        pair = p.split('/')[1]
        print(p, pair.split('_')[0], pair.split('_')[1])
        data = get_data(p, pair.split('_')[0], pair.split('_')[1], authenticated_pair = authenticated_pair)
        all_data = pd.concat([all_data, data])
    return all_data




In [4]:
authenticated_pair = ['S13', 'S21']

all_data = prepare_data('./data', authenticated_pair)
print(all_data.info())

data/S19_S11/I9/S19_S11_I9_T2.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T3.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T1.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T4.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T5.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T7.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T6.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T10.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T8.mat S19 S11
data/S19_S11/I9/S19_S11_I9_T9.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T7.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T6.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T4.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T5.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T1.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T10.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T2.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T3.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T8.mat S19 S11
data/S19_S11/I7/S19_S11_I7_T9.mat S19 S11
data/S19_S11/I6/S19_S11_I6_T10.mat S19 S11
data/S19_S11/I6/S19_S11_I6_T9.mat S19 S11
data/S19_S11/I6/S19_S11_I6_T8.mat S19 S11
data/S19_S11/I6/S19_S11_I6_T5.m

In [5]:
all_data.head()

,timestamp,transmitter,receiver,RSS,A1,A2,A3,A4,A5,A6,...,A22,A23,A24,A25,A26,A27,A28,A29,A30,Authenticated
0,18446744073555091456,S19,S11,41,19.924859,25.059928,25.298221,30.083218,31.144823,37.121422,...,33.837849,38.832976,45.276926,48.270074,46.52956,40.311289,36.359318,35.846897,31.575307,0.0
1,18446744073555093504,S19,S11,40,24.083189,30.0,29.732137,33.837849,40.0,36.345564,...,37.161808,41.761226,49.091751,50.921508,51.478151,43.011626,37.947332,37.054015,35.805028,0.0
2,18446744073555097600,S19,S11,40,23.086793,26.907248,26.400758,35.057096,37.0,35.846897,...,35.846897,37.161808,43.416587,49.396356,46.043458,37.696154,35.0,36.124784,33.837849,0.0
3,18446744073555099648,S19,S11,41,23.430749,24.186773,27.586228,32.310989,38.327536,34.785054,...,35.0,40.804412,46.872167,50.328918,41.182521,39.849718,36.013886,34.928498,31.827661,0.0
4,18446744073555103744,S19,S11,41,26.0,30.083218,31.064449,37.054015,46.669048,48.259714,...,42.43819,48.507731,59.405387,59.439044,58.728187,48.093659,39.446166,46.065171,35.693137,0.0


In [6]:
all_data.loc[all_data['Authenticated'] == 1]

,timestamp,transmitter,receiver,RSS,A1,A2,A3,A4,A5,A6,...,A22,A23,A24,A25,A26,A27,A28,A29,A30,Authenticated
0,561786956,S13,S21,40,28.425341,33.615473,33.42155,46.097722,50.039984,44.384682,...,23.853721,23.021729,21.400935,26.0,24.515301,21.400935,20.248457,21.023796,17.464249,1.0
1,561789812,S13,S21,40,35.014283,32.280025,32.572995,46.389654,47.423623,47.423623,...,18.35756,22.022716,31.144823,27.658633,22.36068,18.681542,24.186773,24.041631,18.601075,1.0
2,561792816,S13,S21,40,31.38471,34.655447,32.202484,43.416587,44.102154,48.020829,...,23.345235,21.023796,27.802878,28.178006,24.020824,22.847319,21.400935,22.203603,16.03122,1.0
3,561798161,S13,S21,41,26.925824,42.190046,41.976184,45.80393,43.931765,48.020829,...,26.248809,26.019224,21.400935,21.954498,29.966648,25.70992,20.124612,16.552945,19.209373,1.0
4,561890477,S13,S21,41,35.355339,31.780497,33.015148,42.107007,50.447993,41.182521,...,15.620499,19.104973,35.014283,30.413813,18.027756,14.56022,23.706539,30.016662,17.464249,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,1736569500,S13,S21,40,20.615528,26.925824,27.658633,33.24154,41.012193,38.897301,...,25.553865,30.675723,27.45906,26.248809,28.178006,22.090722,20.59126,16.278821,15.132746,1.0
1604,1736572517,S13,S21,40,20.615528,30.805844,31.780497,35.171011,41.593269,44.28318,...,30.413813,24.083189,25.0,29.274562,28.84441,21.18962,20.396078,19.313208,16.643317,1.0
1605,1736575572,S13,S21,40,19.026298,34.985711,32.015621,35.510562,42.485292,39.051248,...,25.632011,25.632011,29.154759,27.202941,28.861739,22.671568,16.763055,18.681542,16.155494,1.0
1606,1736579583,S13,S21,40,23.194827,34.539832,36.013886,30.805844,36.796739,45.276926,...,33.955854,32.802439,25.495098,28.635642,32.695565,25.553865,16.124515,17.464249,18.867962,1.0


In [7]:
df = all_data.drop(['timestamp','transmitter', 'receiver','Authenticated'],axis = 1)
dfy = all_data[['Authenticated']]

In [8]:
df.head()
print('----')
dfy.head()

----


,Authenticated
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [9]:
from sklearn.ensemble import RandomForestClassifier

X_train = df
y_train = dfy

regressor = RandomForestClassifier() 
regressor.fit(X_train, y_train)
y_train.shape

/var/folders/1f/c417kfnd55z442_l9_1ktjj40000gn/T/ipykernel_8157/2852920411.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


(331271, 1)

In [10]:
# Save model
import pickle
filename = 'rf_model_class.sav'
pickle.dump(regressor, open(filename, 'wb'))

In [11]:
# print(df.iloc[1])
# test = df.loc[df['Authenticated'] == True]
df_list = df.values.tolist()


In [12]:
test = df_list[40]
print(test)

[41, 23.08679276123039, 26.68332812825267, 32.14031735997639, 38.07886552931954, 39.66106403010388, 36.05551275463989, 35.90264614203248, 34.66987164671943, 33.24154027718932, 34.48187929913333, 35.0, 38.47076812334269, 41.773197148410844, 46.52956049652737, 43.60045871318328, 43.01162633521314, 44.204072210600685, 41.10960958218893, 39.84971769034255, 34.0147027033899, 34.17601498127012, 33.83784863137726, 40.01249804748511, 47.634021455258214, 46.14108798023731, 49.8196748283246, 42.42640687119285, 35.84689665786984, 37.44329045369811, 34.88552708502482]


In [13]:
print(regressor.predict(np.array(test).reshape(1, -1)))


[0.]


/Users/hadi/Desktop/Thesis code/Artificial-Intelligence-for-Security-of-IoT-devices/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [14]:
true = all_data.loc[all_data['Authenticated'] == True]
df_list_true = true.values.tolist()



In [15]:
true_test = df_list_true[0][3:34]
print(true_test)


[40, 28.42534080710379, 33.61547262794322, 33.421549934136806, 46.09772228646444, 50.039984012787215, 44.384682042344295, 45.34313619501854, 35.05709628591621, 34.66987164671943, 37.64306044943742, 36.6742416417845, 37.69615364994153, 40.70626487409524, 42.43819034784589, 42.190046219457976, 41.10960958218893, 34.92849839314596, 31.400636936215164, 27.294688127912362, 24.413111231467404, 25.079872407968907, 23.853720883753127, 23.021728866442675, 21.400934559032695, 26.0, 24.515301344262525, 21.400934559032695, 20.248456731316587, 21.02379604162864, 17.46424919657298]


In [16]:
print(regressor.predict(np.array(true_test).reshape(1, -1)))

[1.]


/Users/hadi/Desktop/Thesis code/Artificial-Intelligence-for-Security-of-IoT-devices/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [17]:
regressor.score(X_train, y_train)

0.9999969813234482

In [18]:
authenticated_pair = ['S13', 'S21']
test_data = prepare_data('./test_data',authenticated_pair = authenticated_pair)

test_data/S19_S11/I12/S19_S11_I12_T10.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T8.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T9.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T1.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T2.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T3.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T7.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T6.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T4.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T5.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T8.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T9.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T10.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T4.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T5.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T7.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T6.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T2.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T3.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T1.mat S19 S11
test_data/S19_S11/

In [19]:
df_test = test_data.drop(['timestamp','transmitter', 'receiver','Authenticated'],axis = 1)
dfy_test = test_data[['Authenticated']]

In [20]:
X_test = df_test
Y_test = dfy_test
Y_test.head()

,Authenticated
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [21]:
Y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104040 entries, 0 to 1621
Data columns (total 1 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Authenticated  104040 non-null  float64
dtypes: float64(1)
memory usage: 1.6 MB


In [22]:
regressor.score(X_test, Y_test)

0.966320645905421

In [23]:
test = df_list[40]
print(test)
print(np.array(test).reshape(1, -1))
print(regressor.predict(np.array(true_test).reshape(1, -1)))


[41, 23.08679276123039, 26.68332812825267, 32.14031735997639, 38.07886552931954, 39.66106403010388, 36.05551275463989, 35.90264614203248, 34.66987164671943, 33.24154027718932, 34.48187929913333, 35.0, 38.47076812334269, 41.773197148410844, 46.52956049652737, 43.60045871318328, 43.01162633521314, 44.204072210600685, 41.10960958218893, 39.84971769034255, 34.0147027033899, 34.17601498127012, 33.83784863137726, 40.01249804748511, 47.634021455258214, 46.14108798023731, 49.8196748283246, 42.42640687119285, 35.84689665786984, 37.44329045369811, 34.88552708502482]
[[41.         23.08679276 26.68332813 32.14031736 38.07886553 39.66106403
  36.05551275 35.90264614 34.66987165 33.24154028 34.4818793  35.
  38.47076812 41.77319715 46.5295605  43.60045871 43.01162634 44.20407221
  41.10960958 39.84971769 34.0147027  34.17601498 33.83784863 40.01249805
  47.63402146 46.14108798 49.81967483 42.42640687 35.84689666 37.44329045
  34.88552709]]
[1.]


/Users/hadi/Desktop/Thesis code/Artificial-Intelligence-for-Security-of-IoT-devices/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
y_pred = regressor.predict(X_test)
df1=pd.DataFrame(data=[[Y_test,y_pred]],columns=['Actual','Predicted'])
print(y_pred)
print(Y_test)

[0. 0. 0. ... 1. 1. 1.]
      Authenticated
0               0.0
1               0.0
2               0.0
3               0.0
4               0.0
...             ...
1617            1.0
1618            1.0
1619            1.0
1620            1.0
1621            1.0

[104040 rows x 1 columns]


In [ ]:
import random
import time
while True:
    rss = [random.randint(40, 90)]
    csi_amplitude = [random.randint(40, 90) for i in range(1,31)]
    test = rss + csi_amplitude
    time.sleep(3)
    print(regressor.predict(np.array(test).reshape(1, -1)) > 0.5)